In [1]:
import os
import shutil

import pandas as pd
import numpy as np
import json 
from os import listdir
from os.path import isfile, join

import plotly
import plotly.express as px
import plotly.io as pio

# nIDR method with threshold computed from null distribution

In [2]:
experiment_id = "NS-23.0061"

metadata_subdir = os.path.expanduser("~/fht.samba.data/experiments/ATACseq/{}/analysis/metadata/".format(experiment_id))
print('metadata_subdir exists:{}\n'.format(os.path.exists(metadata_subdir)))

PEAKS_dir = os.path.expanduser("~/fht.samba.data/experiments/ATACseq/{}/alignment/macs2/".format(experiment_id))
print('PEAKS_dir exists:{}\n'.format(os.path.exists(PEAKS_dir)))

metadata_subdir exists:True

PEAKS_dir exists:True



In [3]:
def create_dir(name_dir):
    if os.path.exists(name_dir):
        shutil.rmtree(name_dir)
    os.mkdir(name_dir)
    return name_dir

In [4]:
merged_narrowPeak_dir = create_dir(name_dir = '../merged_narrowPeak/')
print(merged_narrowPeak_dir)
    
idr_bed_dir = create_dir(name_dir = '../idr_bed/')
print(idr_bed_dir)

../merged_narrowPeak/
../idr_bed/


In [5]:
def read_group_dict(metadata_subdir, experiment_id): 
    output_filename = experiment_id + '_group_dict.json'
    print("output_filename : {}".format(output_filename))

    output_filepath = os.path.join(metadata_subdir, output_filename)
    print("output_filepath : {}".format(output_filepath))
    
    # Opening JSON file
    with open(output_filepath) as json_file:
        contrast_dict = json.load(json_file)
        
    return contrast_dict

group_dict = read_group_dict(metadata_subdir, experiment_id)
group_dict

output_filename : NS-23.0061_group_dict.json
output_filepath : /home/fuzan/fht.samba.data/experiments/ATACseq/NS-23.0061/analysis/metadata/NS-23.0061_group_dict.json


{'WT_HCT116': ['SRR5876158', 'SRR5876159'],
 'ARID1BKD_HCT116': ['SRR5876160', 'SRR5876161'],
 'ARID1AKO_HCT116': ['SRR5876162', 'SRR5876163'],
 'ARID1AKO_ARID1BKD_HCT116': ['SRR5876164', 'SRR5876165'],
 'WT_TOV21G': ['SRR5876661', 'SRR5876662'],
 'ARID1BKD_TOV21G': ['SRR5876663', 'SRR5876664']}

In [6]:
def combine_narrowPeak_files(group):    

    # write path of each narrowPeak file
    narrowPeak_list = [PEAKS_dir + s + '_peaks.narrowPeak' for s in group_dict[group]]

    # define variable name for each replicate file
    narrowPeak_list_bash = '\t'.join(narrowPeak_list)
    
    # combined 3 replicates
    !grep -h '^chr'  {narrowPeak_list_bash} > {merged_narrowPeak_dir}{group}.combined.narrowPeak

    # sort reads by chromosome coordinates
    !sort -k1,1V -k2,2n -k3,3n {merged_narrowPeak_dir}{group}.combined.narrowPeak > {merged_narrowPeak_dir}{group}.sorted.narrowPeak

    # bedtools merge reads
    !bedtools merge -d 50 -c 4,8 -o collapse,collapse -delim "|" -i {merged_narrowPeak_dir}{group}.sorted.narrowPeak > {merged_narrowPeak_dir}{group}.all_merged.narrowPeak

    # reformat replicate column
    cmd_format = """awk '{{ gsub(/_peak_[a-z0-9]*/, "", $4) }}1' {merged_narrowPeak_dir}{group}.all_merged.narrowPeak >  {merged_narrowPeak_dir}{group}.merged.narrowPeak""".format(merged_narrowPeak_dir=merged_narrowPeak_dir, group=group)
    ! {cmd_format}

In [7]:
def format_pivot_merged_narrowPeak_mean(merged_narrowPeak_dir, group):
    #  Read merged dataframe + format
    merged_narrowPeak = pd.read_table(merged_narrowPeak_dir + group + '.merged.narrowPeak', delimiter = ' ', header=None)
    merged_narrowPeak.columns = ['chr', 'start', 'end', 'peak', 'logFC']
    merged_narrowPeak['peak_id'] = merged_narrowPeak['chr'] + ':'+ merged_narrowPeak['start'].astype('str')  + '-' + merged_narrowPeak['end'].astype('str')
    merged_narrowPeak.set_index("peak_id", inplace = True)

    # split peak_id and logFC
    merged_narrowPeak["peak_split"] = merged_narrowPeak.peak.str.split("|")
    merged_narrowPeak["logFC_split"] = merged_narrowPeak.logFC.str.split("|")

    # create one column with peak_id and logFC tuple
    merged_narrowPeak["peak_logFC_zip"] = merged_narrowPeak.apply(lambda row: list(zip(row.peak_split, row.logFC_split)), axis=1)

    # create a row for each tuple
    long_merged_narrowPeak = merged_narrowPeak[["chr", "start", "end", "peak_logFC_zip"]].explode("peak_logFC_zip")

    # split tuples into two columns
    long_merged_narrowPeak["sample_ID"] = [x[0] for x in long_merged_narrowPeak.peak_logFC_zip]
    long_merged_narrowPeak["logFC"] = [float(x[1]) for x in long_merged_narrowPeak.peak_logFC_zip]

    # compute mean of logFC for each peak_id and replicate
    merged_narrowPeak_mean = long_merged_narrowPeak.reset_index().drop("peak_logFC_zip", axis=1).groupby(["peak_id", "chr", "start", "end", "sample_ID"], sort=False).mean().reset_index().set_index("peak_id")

    # pivot sample_id column into three columns
    pivot_merged_narrowPeak_mean= merged_narrowPeak_mean[["logFC", "sample_ID"]].reset_index().pivot(index="peak_id", columns="sample_ID", values="logFC").fillna(0)
    
    return  merged_narrowPeak_mean, pivot_merged_narrowPeak_mean

In [8]:
def generate_shuffled_min_perc_rank(pivot_merged_narrowPeak_mean):
    shuffled_pivot_merged_narrowPeak_mean = pivot_merged_narrowPeak_mean.copy()#.sample(frac = 1, axis=0)
    for col in shuffled_pivot_merged_narrowPeak_mean.columns:
        t = shuffled_pivot_merged_narrowPeak_mean[col].to_numpy()
        np.random.shuffle(t)
        shuffled_pivot_merged_narrowPeak_mean[col] = t
    
    # compute min of percentage rank of mean for each replicate
    shuffled_min_perc_rank =  shuffled_pivot_merged_narrowPeak_mean.rank(pct=True, axis=0).min(axis=1)
    return shuffled_min_perc_rank

In [9]:
def compute_threshold(shuffled_min_perc_rank, group):
    shuffled_sorted_min_perc_rank = shuffled_min_perc_rank.iloc[::10].sort_values()   
    # fig = px.ecdf(shuffled_sorted_min_perc_rank)
    shuffled_overall_min_perc_rank = np.linspace(0., 1., num=shuffled_sorted_min_perc_rank.shape[0])

    # find index from shuffled_sorted_min_perc_rank that has the closest number to .9 in shuffled_overall_min_perc_rank 
    index = np.argmin(np.abs(np.array(shuffled_overall_min_perc_rank)-.9))
    threshold = shuffled_sorted_min_perc_rank[index]
    print('\n% of rank min the closest to 0.9: \n{}'.format(shuffled_overall_min_perc_rank[index]))
    print('\nrank min corresponding to 10% of kept reads, i.e. new threshold: \n{}'.format(threshold))
    
    # fig = px.scatter(x=shuffled_sorted_min_perc_rank,y=shuffled_overall_min_perc_rank, title=group)
    # fig.add_vline(x=threshold,  line_width=3, line_dash="dash", line_color="green")
    # fig.show()
    
    return threshold

In [10]:
def filter_merged_narrowPeak(pivot_merged_narrowPeak_mean, merged_narrowPeak_mean, threshold, group):
    # compute min of percentage rank of mean for each replicate
    min_perc_rank =  pivot_merged_narrowPeak_mean.rank(pct=True, axis=0).min(axis=1)
    
    # # plot ECDF
    # sorted_min_perc_rank = min_perc_rank.iloc[::10].sort_values()
    # overall_min_perc_rank = np.linspace(0., 1., num=sorted_min_perc_rank.shape[0])
    # fig = px.scatter(x=sorted_min_perc_rank,y=overall_min_perc_rank, title=group)
    # fig.add_vline(x=threshold,  line_width=3, line_dash="dash", line_color="green")
    # fig.show()

    # get list of peaks that we keep (min rank > threshold)
    keep_peaks = min_perc_rank.index[min_perc_rank > threshold]

    # filter narrowPeak with list of peaks
    filt_IDR_narrowPeak = merged_narrowPeak_mean.loc[keep_peaks]

    # drop sample_ID and logFC columns to delete duplicated peak_id
    filt_IDR_narrowPeak.drop(['sample_ID', 'logFC'], axis=1, inplace=True)

    # remove duplicates
    filt_IDR_narrowPeak.drop_duplicates(inplace=True)
    
    print("pivot_merged_narrowPeak_mean.shape:  {}".format(pivot_merged_narrowPeak_mean.shape))
    print("filt_IDR_narrowPeak.shape:  {}".format(filt_IDR_narrowPeak.shape))
    return filt_IDR_narrowPeak

In [11]:
def create_narrowPeak(group_dict):
    
    group_list = list(group_dict.keys())
    print('group_list: {}'.format(group_list))

    for group in group_list:

        print('\ngroup: {}'.format(group))
        
        combine_narrowPeak_files(group)
        merged_narrowPeak_mean, pivot_merged_narrowPeak_mean = format_pivot_merged_narrowPeak_mean(merged_narrowPeak_dir, group)
        shuffled_min_perc_rank = generate_shuffled_min_perc_rank(pivot_merged_narrowPeak_mean)
        threshold = compute_threshold(shuffled_min_perc_rank, group)
        filt_IDR_narrowPeak = filter_merged_narrowPeak(pivot_merged_narrowPeak_mean, merged_narrowPeak_mean, threshold, group)
        
        # Plot ECDF
        shuffled_sorted_min_perc_rank = shuffled_min_perc_rank.iloc[::10].sort_values()
        shuffled_overall_min_perc_rank = np.linspace(0., 1., num=shuffled_sorted_min_perc_rank.shape[0])
        fig = px.scatter(x=shuffled_sorted_min_perc_rank,y=shuffled_overall_min_perc_rank, title=group)
        fig.add_vline(x=threshold,  line_width=3, line_dash="dash", line_color="green")


        min_perc_rank =  pivot_merged_narrowPeak_mean.rank(pct=True, axis=0).min(axis=1)
        sorted_min_perc_rank = min_perc_rank.iloc[::10].sort_values()
        overall_min_perc_rank = np.linspace(0., 1., num=sorted_min_perc_rank.shape[0])
        fig.add_scatter(x=sorted_min_perc_rank, y=overall_min_perc_rank, mode="markers", name="real")
        fig.update_layout(xaxis_title='min percent rank')
        fig.update_layout(yaxis_title='% of peaks')
        # fig.show()

        # export as static image
        pio.write_image(fig, merged_narrowPeak_dir + group + "_ECDF.png")
                
        # Save IDR narrowPeak
        filt_IDR_narrowPeak.to_csv(idr_bed_dir + group + '.IDR.narrowPeak', sep="\t", index=None, header=None)
        
create_narrowPeak(group_dict)  

group_list: ['WT_HCT116', 'ARID1BKD_HCT116', 'ARID1AKO_HCT116', 'ARID1AKO_ARID1BKD_HCT116', 'WT_TOV21G', 'ARID1BKD_TOV21G']

group: WT_HCT116

% of rank min the closest to 0.9: 
0.8999999999999999

rank min corresponding to 10% of kept reads, i.e. new threshold: 
0.6519628042192422
pivot_merged_narrowPeak_mean.shape:  (308207, 2)
filt_IDR_narrowPeak.shape:  (60489, 3)

group: ARID1BKD_HCT116

% of rank min the closest to 0.9: 
0.9000052532044547

rank min corresponding to 10% of kept reads, i.e. new threshold: 
0.6815840621963071
pivot_merged_narrowPeak_mean.shape:  (380730, 2)
filt_IDR_narrowPeak.shape:  (68550, 3)

group: ARID1AKO_HCT116

% of rank min the closest to 0.9: 
0.9

rank min corresponding to 10% of kept reads, i.e. new threshold: 
0.717031717106752
pivot_merged_narrowPeak_mean.shape:  (419805, 2)
filt_IDR_narrowPeak.shape:  (60139, 3)

group: ARID1AKO_ARID1BKD_HCT116

% of rank min the closest to 0.9: 
0.8999899989999001

rank min corresponding to 10% of kept reads, i.e. 